In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import time

# Cihazı ayarlayın (GPU varsa CUDA, yoksa CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)

# Veriyi yükle
file_path = "C:/Users/alica/Desktop/300k_data.txt"
data = pd.read_csv(file_path, header=None, names=["M", "Peri", "Node", "Incl.", "E", "N", "A", "Epoch"], sep="_")

# Gelecekteki pozisyonu (Future_Position) rastgele değerlerle dolduralım
data["Future_Position"] = np.random.uniform(2.0, 3.5, size=len(data))

# Özellikleri (features) ve hedef değeri (target) ayır
X = data[["M", "Peri", "Node", "Incl.", "E", "N", "A", "Epoch"]].values
y = data["Future_Position"].values

# Veriyi eğitim, doğrulama ve test setlerine bölme (%70 Eğitim, %20 Doğrulama, %10 Test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)  # %70 Eğitim
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)  # %20 Doğrulama, %10 Test

# Veriyi ölçeklendir
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Veriyi PyTorch tensörlerine dönüştür ve GPU'ya taşı
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# PyTorch DataLoader ile veri kümelerini oluştur
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=512, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=512, num_workers=0, pin_memory=True)

# Modeli oluştur
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = SimpleNN().to(device)

# Optimizasyon ve düzenlileştirme
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-5)

# Eğitim döngüsü
epochs = 100
start_time = time.time()
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    # Scheduler adımı
    scheduler.step()

    # Doğrulama seti üzerinde değerlendirme
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, targets).item()
    val_loss /= len(val_loader)

    # Her %10 ilerlemede çıktı
    if (epoch + 1) % (epochs // 10) == 0:
        print(f"%{(epoch + 1) / epochs * 100:.0f} tamamlandı - Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

end_time = time.time()
print(f"Eğitim süresi: {end_time - start_time:.2f} saniye")

# Test seti üzerinde değerlendirme
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        test_loss += criterion(outputs, targets).item()
test_loss /= len(test_loader)
print(f"Test seti üzerinde ortalama kayıp değeri: {test_loss:.4f}")


Kullanılan cihaz: cuda
%10 tamamlandı - Epoch [10/100], Train Loss: 0.1875, Val Loss: 0.1898
%20 tamamlandı - Epoch [20/100], Train Loss: 0.1888, Val Loss: 0.1937
%30 tamamlandı - Epoch [30/100], Train Loss: 0.1867, Val Loss: 0.1893
%40 tamamlandı - Epoch [40/100], Train Loss: 0.1883, Val Loss: 0.1922
%50 tamamlandı - Epoch [50/100], Train Loss: 0.1864, Val Loss: 0.1894
%60 tamamlandı - Epoch [60/100], Train Loss: 0.1880, Val Loss: 0.1904
%70 tamamlandı - Epoch [70/100], Train Loss: 0.1862, Val Loss: 0.1893
%80 tamamlandı - Epoch [80/100], Train Loss: 0.1878, Val Loss: 0.1913
%90 tamamlandı - Epoch [90/100], Train Loss: 0.1861, Val Loss: 0.1894
%100 tamamlandı - Epoch [100/100], Train Loss: 0.1877, Val Loss: 0.1896
Eğitim süresi: 198.79 saniye
Test seti üzerinde ortalama kayıp değeri: 0.1876


In [ ]:
from datetime import datetime
from astropy.time import Time

# Tarihi datetime formatında oluştur
date = datetime(2024, 11, 25)
dateSecond = datetime(2024,12,1)

# Julian Date hesapla
jd1 = Time(date, scale='utc').jd
jd2 = Time(dateSecond, scale= "utc").jd

print(f"Julian Date (UTC): {jd1}")

print(jd2-jd1)

Julian Date (UTC): 2460639.5
6.0


In [3]:
import math
from datetime import datetime

# Kepler denklemini çözmek için bir fonksiyon
def solve_kepler(M, e, tol=1e-6):
    E = M  # İlk tahmin
    while True:
        delta_E = (E - e * math.sin(E) - M) / (1 - e * math.cos(E))
        E -= delta_E
        if abs(delta_E) < tol:
            break
    return E

def calculate_position(M0, Peri, Node, Incl, e, n, a, epoch_date, target_date):
    # Zaman farkını hesapla (gün cinsinden)
    delta_t = (target_date - epoch_date).days

    # Ortalama Anomali (M)
    M = math.radians(M0 + n * delta_t)  # Dereceden radyana

    # Kepler Denklemi ile Eksen Anomali (E)
    E = solve_kepler(M, e)

    # Gerçek Anomali (nu)
    nu = 2 * math.atan2(
        math.sqrt(1 + e) * math.sin(E / 2),
        math.sqrt(1 - e) * math.cos(E / 2)
    )

    # Heliocentric Distance (r) - AU cinsinde bırakıyoruz
    r = a * (1 - e * math.cos(E))

    # Yörünge Düzlemindeki Konum (x', y')
    x_prime = r * math.cos(nu)
    y_prime = r * math.sin(nu)

    # Ekliptik Koordinatlar
    Peri = math.radians(Peri)
    Node = math.radians(Node)
    Incl = math.radians(Incl)

    x = (
        x_prime * (math.cos(Node) * math.cos(Peri) - math.sin(Node) * math.sin(Peri) * math.cos(Incl)) -
        y_prime * (math.cos(Node) * math.sin(Peri) + math.sin(Node) * math.cos(Peri) * math.cos(Incl))
    )
    y = (
        x_prime * (math.sin(Node) * math.cos(Peri) + math.cos(Node) * math.sin(Peri) * math.cos(Incl)) -
        y_prime * (math.sin(Node) * math.sin(Peri) - math.cos(Node) * math.cos(Peri) * math.cos(Incl))
    )
    z = (
        x_prime * (math.sin(Peri) * math.sin(Incl)) +
        y_prime * (math.cos(Peri) * math.sin(Incl))
    )

    return x, y, z, r

def process_file(input_file, output_file, target_date):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    results = []
    for line in lines:
        # Verileri ayrıştır (split('_') kullan)
        data = line.strip().split('_')
        M0 = float(data[0])
        Peri = float(data[1])
        Node = float(data[2])
        Incl = float(data[3])
        e = float(data[4])
        n = float(data[5])
        a = float(data[6])
        epoch_date = datetime.strptime(data[7], '%Y%m%d')

        # Epoch tarihindeki konumu hesapla
        x_epoch, y_epoch, z_epoch, r_epoch = calculate_position(M0, Peri, Node, Incl, e, n, a, epoch_date, epoch_date)

        # Hedef tarihteki konumu hesapla
        x_target, y_target, z_target, r_target = calculate_position(M0, Peri, Node, Incl, e, n, a, epoch_date, target_date)

        # Sonuçları formatla (yalnızca hedef tarihteki x, y, z değerleri)
        result_line = f"{x_target:.6f}_{y_target:.6f}_{z_target:.6f}\n"
        results.append(result_line)

    # Sonuçları yaz
    with open(output_file, 'w') as file:
        file.writelines(results)

if __name__ == "__main__":
    input_file = "C:/Users/alica/Desktop/200_data.txt"
    output_file = "C:/Users/alica/Desktop/resultxx.txt"
    target_date_str = input("Enter target date (YYYYMMDD): ")
    target_date = datetime.strptime(target_date_str, '%Y%m%d')

    process_file(input_file, output_file, target_date)
    print(f"Results saved to {output_file}")


Results saved to C:/Users/alica/Desktop/resultxx.txt


In [1]:
import math
from datetime import datetime

# Kepler denklemini çözmek için bir fonksiyon
def solve_kepler(M, e, tol=1e-6):
    E = M  # İlk tahmin
    while True:
        delta_E = (E - e * math.sin(E) - M) / (1 - e * math.cos(E))
        E -= delta_E
        if abs(delta_E) < tol:
            break
    return E

def calculate_position(M0, Peri, Node, Incl, e, n, a, epoch_date):
    # Ortalama Anomali (M)
    M = math.radians(M0)  # Epoch tarihinde M0 kullanılır, zaman farkı sıfırdır.

    # Kepler Denklemi ile Eksen Anomali (E)
    E = solve_kepler(M, e)

    # Gerçek Anomali (nu)
    nu = 2 * math.atan2(
        math.sqrt(1 + e) * math.sin(E / 2),
        math.sqrt(1 - e) * math.cos(E / 2)
    )

    # Heliocentric Distance (r) - AU cinsinde bırakıyoruz
    r = a * (1 - e * math.cos(E))

    # Yörünge Düzlemindeki Konum (x', y')
    x_prime = r * math.cos(nu)
    y_prime = r * math.sin(nu)

    # Ekliptik Koordinatlar
    Peri = math.radians(Peri)
    Node = math.radians(Node)
    Incl = math.radians(Incl)

    x = (
        x_prime * (math.cos(Node) * math.cos(Peri) - math.sin(Node) * math.sin(Peri) * math.cos(Incl)) -
        y_prime * (math.cos(Node) * math.sin(Peri) + math.sin(Node) * math.cos(Peri) * math.cos(Incl))
    )
    y = (
        x_prime * (math.sin(Node) * math.cos(Peri) + math.cos(Node) * math.sin(Peri) * math.cos(Incl)) -
        y_prime * (math.sin(Node) * math.sin(Peri) - math.cos(Node) * math.cos(Peri) * math.cos(Incl))
    )
    z = (
        x_prime * (math.sin(Peri) * math.sin(Incl)) +
        y_prime * (math.cos(Peri) * math.sin(Incl))
    )

    return x, y, z

def process_file(input_file, output_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    results = []
    for line in lines:
        # Verileri ayrıştır (split('_') kullan)
        data = line.strip().split('_')
        M0 = float(data[0])
        Peri = float(data[1])
        Node = float(data[2])
        Incl = float(data[3])
        e = float(data[4])
        n = float(data[5])
        a = float(data[6])
        epoch_date = datetime.strptime(data[7], '%Y%m%d')

        # Epoch tarihindeki konumu hesapla
        x_epoch, y_epoch, z_epoch = calculate_position(M0, Peri, Node, Incl, e, n, a, epoch_date)

        # Sonuçları formatla
        result_line = f"{line.strip()}_{x_epoch:.6f}_{y_epoch:.6f}_{z_epoch:.6f}\n"
        results.append(result_line)

    # Sonuçları yaz
    with open(output_file, 'w') as file:
        file.writelines(results)

if __name__ == "__main__":
    input_file = "C:/Users/alica/Desktop/200_data.txt"
    output_file = "C:/Users/alica/Desktop/200_first.txt"

    process_file(input_file, output_file)
    print(f"Results saved to {output_file}")


Results saved to C:/Users/alica/Desktop/200_first.txt


In [2]:
import math
from datetime import datetime

# Kepler denklemini çözmek için bir fonksiyon
def solve_kepler(M, e, tol=1e-6):
    E = M  # İlk tahmin
    while True:
        delta_E = (E - e * math.sin(E) - M) / (1 - e * math.cos(E))
        E -= delta_E
        if abs(delta_E) < tol:
            break
    return E

def calculate_position(M0, Peri, Node, Incl, e, n, a, epoch_date, target_date):
    # Zaman farkını hesapla (gün cinsinden)
    delta_t = (target_date - epoch_date).days

    # Ortalama Anomali (M)
    M = math.radians(M0 + n * delta_t)  # Dereceden radyana

    # Kepler Denklemi ile Eksen Anomali (E)
    E = solve_kepler(M, e)

    # Gerçek Anomali (nu)
    nu = 2 * math.atan2(
        math.sqrt(1 + e) * math.sin(E / 2),
        math.sqrt(1 - e) * math.cos(E / 2)
    )

    # Heliocentric Distance (r) - AU cinsinde bırakıyoruz
    r = a * (1 - e * math.cos(E))

    # Yörünge Düzlemindeki Konum (x', y')
    x_prime = r * math.cos(nu)
    y_prime = r * math.sin(nu)

    # Ekliptik Koordinatlar
    Peri = math.radians(Peri)
    Node = math.radians(Node)
    Incl = math.radians(Incl)

    x = (
        x_prime * (math.cos(Node) * math.cos(Peri) - math.sin(Node) * math.sin(Peri) * math.cos(Incl)) -
        y_prime * (math.cos(Node) * math.sin(Peri) + math.sin(Node) * math.cos(Peri) * math.cos(Incl))
    )
    y = (
        x_prime * (math.sin(Node) * math.cos(Peri) + math.cos(Node) * math.sin(Peri) * math.cos(Incl)) -
        y_prime * (math.sin(Node) * math.sin(Peri) - math.cos(Node) * math.cos(Peri) * math.cos(Incl))
    )
    z = (
        x_prime * (math.sin(Peri) * math.sin(Incl)) +
        y_prime * (math.cos(Peri) * math.sin(Incl))
    )

    return x, y, z

def process_file(input_file, output_file, target_date):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    results = []
    for line in lines:
        # Verileri ayrıştır (split('_') kullan)
        data = line.strip().split('_')
        M0 = float(data[0])
        Peri = float(data[1])
        Node = float(data[2])
        Incl = float(data[3])
        e = float(data[4])
        n = float(data[5])
        a = float(data[6])
        epoch_date = datetime.strptime(data[7], '%Y%m%d')

        # Hedef tarihteki konumu hesapla
        x_target, y_target, z_target = calculate_position(M0, Peri, Node, Incl, e, n, a, epoch_date, target_date)

        # Sonuçları formatla (sadece x, y, z)
        result_line = f"{x_target:.6f}_{y_target:.6f}_{z_target:.6f}\n"
        results.append(result_line)

    # Sonuçları yaz
    with open(output_file, 'w') as file:
        file.writelines(results)

if __name__ == "__main__":
    input_file = "C:/Users/alica/Desktop/200_data.txt"
    output_file = "C:/Users/alica/Desktop/200_data_second.txt"
    target_date_str = input("Enter target date (YYYYMMDD): ")
    target_date = datetime.strptime(target_date_str, '%Y%m%d')

    process_file(input_file, output_file, target_date)
    print(f"Results saved to {output_file}")


Results saved to C:/Users/alica/Desktop/200_data_second.txt


In [51]:
import pandas as pd

# Dosya yolu ve çıktı dosyası
file_path = "C:/Users/alica/Desktop/comparison_results_fixed.csv"  # Giriş dosyasının yolu
output_file = "C:/Users/alica/Desktop/caglar.csv"  # Çıktı dosyasının adı

# Verileri yükle
data = pd.read_csv(file_path)

# Farkları hesapla ve yüzdelik farkları bul
data['x_diff'] = ((data['x_pred'] - data['x_true']) / data['x_true']) * 100
data['y_diff'] = ((data['y_pred'] - data['y_true']) / data['y_true']) * 100
data['z_diff'] = ((data['z_pred'] - data['z_true']) / data['z_true']) * 100

# NaN ve sonsuz değerleri temizle
data['x_diff'] = data['x_diff'].replace([float('inf'), -float('inf')], 0).fillna(0)
data['y_diff'] = data['y_diff'].replace([float('inf'), -float('inf')], 0).fillna(0)
data['z_diff'] = data['z_diff'].replace([float('inf'), -float('inf')], 0).fillna(0)

# Ortalama yüzdelik fark hesaplama
data['average_percent'] = (
    data[['x_diff', 'y_diff', 'z_diff']].mean(axis=1)
)

# %70'ten büyük hata paylarını tespit et
high_error_rows = data[data['average_percent'].abs() > 70]
filtered_data = data[data['average_percent'].abs() <= 70]  # Genel hata için filtrelenmiş veriler

# %70'ten büyük hata paylarının sayısını bul
high_error_count = len(high_error_rows)
print(f"%70'den büyük hata payına sahip {high_error_count} satır bulundu.")

# Genel ortalama hata yüzdesini filtrelenmiş verilerle hesapla
if len(filtered_data) > 0:
    total_average_error = (
        filtered_data[['x_diff', 'y_diff', 'z_diff']].abs().sum().sum() / (len(filtered_data) * 3)
    )
    print(f"Genel Ortalama Hata Payı (filtrelenmiş): {total_average_error:.1f}%")
else:
    print("Filtrelenmiş veri bulunmamaktadır, genel hata payı hesaplanamadı.")

# Ortalama yüzdelik farkları tam sayı ve bir ondalık hane olacak şekilde yuvarla
data['average_percent'] = data['average_percent'].round(1).astype(str) + "%"

# Sadece `average_percent` sütununu kaydet
average_percentage = data[['average_percent']]
average_percentage.to_csv(output_file, index=False, header=False)

print(f"Sadece ortalama yüzdelik farklarla dosya '{output_file}' olarak kaydedildi.")


%70'den büyük hata payına sahip 810 satır bulundu.
Genel Ortalama Hata Payı (filtrelenmiş): 16.7%
Sadece ortalama yüzdelik farklarla dosya 'C:/Users/alica/Desktop/caglar.csv' olarak kaydedildi.


In [52]:
import pandas as pd

# Dosya yolu ve çıktı dosyası
file_path = "C:/Users/alica/Desktop/comparison_results_fixed.csv"  # Giriş dosyasının yolu
# Verileri yükle
data = pd.read_csv(file_path)

# Farkları hesapla ve yüzdelik farkları bul
data['x_diff'] = ((data['x_pred'] - data['x_true']) / data['x_true']) * 100
data['y_diff'] = ((data['y_pred'] - data['y_true']) / data['y_true']) * 100
data['z_diff'] = ((data['z_pred'] - data['z_true']) / data['z_true']) * 100

# NaN ve sonsuz değerleri temizle
data['x_diff'] = data['x_diff'].replace([float('inf'), -float('inf')], 0).fillna(0)
data['y_diff'] = data['y_diff'].replace([float('inf'), -float('inf')], 0).fillna(0)
data['z_diff'] = data['z_diff'].replace([float('inf'), -float('inf')], 0).fillna(0)

# Ortalama yüzdelik fark hesaplama
data['average_percent'] = (
    data[['x_diff', 'y_diff', 'z_diff']].mean(axis=1)
)

# %70'ten büyük hata paylarını tespit et
high_error_rows = data[data['average_percent'].abs() > 70]
filtered_data = data[data['average_percent'].abs() <= 70]  # Genel hata için filtrelenmiş veriler

# %70'ten büyük hata paylarının sayısını bul
high_error_count = len(high_error_rows)
print(f"%70'den büyük hata payına sahip {high_error_count} satır bulundu.")

# Genel ortalama hata yüzdesini filtrelenmiş verilerle hesapla
if len(filtered_data) > 0:
    total_average_error = (
        filtered_data[['x_diff', 'y_diff', 'z_diff']].abs().sum().sum() / (len(filtered_data) * 3)
    )
    print(f"Genel Ortalama Hata Payı (filtrelenmiş): {total_average_error:.1f}%")
else:
    print("Filtrelenmiş veri bulunmamaktadır, genel hata payı hesaplanamadı.")

# Ortalama yüzdelik farkları tam sayı ve bir ondalık hane olacak şekilde yuvarla
data['average_percent'] = data['average_percent'].round(1).astype(str) + "%"



%70'den büyük hata payına sahip 29922 satır bulundu.
Genel Ortalama Hata Payı (filtrelenmiş): 19.4%
